In [3]:
%load_ext autoreload
%autoreload 2

from boolrank import DualSiglip2Model
from evaluation import evaluate_on_generated

# model = DualSiglip2Model('BAAI/llm-embedder')
model = DualSiglip2Model('BAAI/bge-small-en-v1.5')
# model = DualSiglip2Model('dmis-lab/biobert-v1.1')
_ = model.eval()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pandas as pd
from collections import defaultdict
import torch

group_keys: list[str] = ["id", "model"]
main_name = group_keys[-1]
res = defaultdict(lambda: defaultdict(list))
df = pd.read_json("data/combined_outputs.jsonl", lines=True)
byid = df.sort_values("f3").groupby(group_keys)
prompt_data = list(map(lambda tpl: tpl[1], byid))

group = prompt_data[0]
print(group.f3)
logits = model(list(group["generated_query"].values), group["topic"].iloc[0])["logits"]
tensor = logits.squeeze().cpu()
print(tensor)
torch.argsort(tensor)

1266    0.0000
1380    0.0277
1283    0.0308
1179    0.0314
1219    0.0435
1322    0.0741
1139    0.0856
Name: f3, dtype: float64
tensor([0.7545, 0.7496, 0.7905, 0.7594, 0.7945, 0.7015, 0.7695],
       grad_fn=<ToCopyBackward0>)


tensor([5, 1, 0, 3, 6, 2, 4])

## Untrained

In [2]:
evaluate_on_generated(model)

NameError: name 'evaluate_on_generated' is not defined

In [3]:
evaluate_on_generated(model, ["id"])

Evaluating groups: 100%|██████████| 40/40 [00:26<00:00,  1.49it/s]
<string>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.


,id,spearman,pearson,f3_variance,best_rank
0,1,-0.2038,-0.2442,0.0238,0.0000
1,2,-0.0807,-0.0631,0.0280,0.0000
2,3,0.2340,0.2122,0.1361,0.5405
3,4,0.2586,0.2930,0.0395,0.5172
4,6,-0.1980,-0.2421,0.0964,0.0270
5,7,-0.3480,-0.4912,0.0522,0.5833
6,8,-0.3210,-0.2015,0.0380,0.1538
7,10,0.5156,0.4624,0.1157,0.7750
8,11,0.0023,-0.2777,0.1264,0.8182
9,12,0.2039,0.1940,0.1099,0.0952


## Model 1

In [ ]:
model.load(r"models\clip\bge-small-en-v1.5\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# model.load(r"models\clip\biobert-v1.1\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-14110\model.safetensors")
evaluate_on_generated(model)

DualSiglip2Model(
  (encoder_bool): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, 

In [20]:
evaluate_on_generated(model, ["id"])

Evaluating groups: 100%|██████████| 40/40 [00:28<00:00,  1.40it/s]
<string>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.


,id,spearman,pearson,f3_variance
0,1,0.1499,0.1373,0.0238
1,2,-0.4789,-0.5184,0.0280
2,3,-0.0287,-0.0319,0.1361
3,4,-0.1227,0.1435,0.0395
4,6,0.1769,0.1084,0.0964
5,7,0.1902,0.0789,0.0522
6,8,0.1651,0.0428,0.0380
7,10,-0.2105,0.1164,0.1157
8,11,-0.0531,-0.1159,0.1264
9,12,-0.2714,-0.2478,0.1099


## Model 2

In [21]:
model.load(r"models\clip\bge-small-en-v1.5\b16_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# evaluate_on_generated(model)

DualSiglip2Model(
  (encoder_bool): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, 

In [23]:
evaluate_on_generated(model, ["id"])

Evaluating groups: 100%|██████████| 40/40 [00:27<00:00,  1.46it/s]
<string>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.


,id,spearman,pearson,f3_variance
0,1,0.0718,0.0138,0.0238
1,2,-0.3333,-0.3502,0.0280
2,3,0.0702,-0.0122,0.1361
3,4,0.1197,0.1119,0.0395
4,6,0.0650,0.0671,0.0964
5,7,-0.1616,-0.4852,0.0522
6,8,-0.4660,-0.2771,0.0380
7,10,0.5062,0.4091,0.1157
8,11,0.0739,-0.1904,0.1264
9,12,0.3896,0.4052,0.1099
